In [2]:
from langchain.embeddings.base import Embeddings
from tokenizers import Tokenizer
import onnxruntime as ort
import numpy as np


class OnnxEmbedder(Embeddings):
  def __init__(self, model_path: str, tokenizer_path: str):
      self.session = ort.InferenceSession(model_path)
      self.tokenizer = Tokenizer.from_file(tokenizer_path)

  def _embed(self, text: str):
      encoding = self.tokenizer.encode(text)
      input_ids = np.array([encoding.ids], dtype=np.int64)
      attention_mask = np.array([[1] * len(encoding.ids)], dtype=np.int64)

      outputs = self.session.run(
          None, {"input_ids": input_ids, "attention_mask": attention_mask}
      )

      raw_vector = outputs[0][0]
      norm_vector = raw_vector / (np.linalg.norm(raw_vector) + 1e-10)
      return norm_vector.tolist()

  def embed_documents(self, texts: list[str]) -> list[list[float]]:
      return [self._embed(text) for text in texts]

  def embed_query(self, text: str) -> list[float]:
      return self._embed(text)

In [3]:
import pandas as pd

df_raw = pd.read_csv("../../db/news_2023_2025_raw.csv")
df_raw.head(5)

df_metadata = pd.read_csv("../../db/news_2023_2025_metadata2.csv")
df_metadata.head(5)

df_data = pd.merge(df_raw, df_metadata, on="news_id")
df_data.head(5)

,news_id,wdate,title,article,press,url,image,summary,stock_list,stock_list_view,industry_list,impact_score
0,20250523_0002,2025-05-23 18:52:00,"[단독] 카카오페이, 2500만 회원 쓱·스마일페이 품나…간편결제 시장 빅3 경쟁 후끈",매각가 5000억 안팎 달할듯\n결제시장 내 입지강화 포석\n카카오페이 [사진 = ...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 국내 대표 전자결제사업자인 카...,"[{""stock_id"": ""377300"", ""stock_name"": ""카카오페이""}]","[{""stock_id"": ""377300"", ""stock_name"": ""카카오페이""}]","[{""stock_id"": ""377300"", ""industry_id"": ""116601...",0.33
1,20250523_0004,2025-05-23 18:33:00,"골드만삭스 차기 CEO, 이재용·김병주·이창용 만났다",[사진=연합뉴스]\n세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...,세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자(COO)인 존 월드...,"[{""stock_id"": ""005930"", ""stock_name"": ""삼성전자""}]","[{""stock_id"": ""005930"", ""stock_name"": ""삼성전자""}]","[{""stock_id"": ""005930"", ""industry_id"": ""32604""...",0.05
2,20250523_0007,2025-05-23 18:00:00,[단독] 전자결제 강자 카카오페이 쓱·스마일 페이 인수 추진,국내 대표 전자결제사업자인 카카오페이가 신세계이마트 산하 간편결제사업부 인수에 나섰...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://ssl.pstatic.net/static.news/image/news...,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 카카오페이가 SSG닷컴 쓱페이...,"[{""stock_id"": ""377300"", ""stock_name"": ""카카오페이""}]","[{""stock_id"": ""377300"", ""stock_name"": ""카카오페이""}]","[{""stock_id"": ""377300"", ""industry_id"": ""116601...",0.33
3,20250523_0010,2025-05-23 17:52:00,조현준 효성重 지분 4.9% 美 테크펀드 2600억 매각,조현준 효성 회장이 상속세 재원 마련을 위해 효성중공업 지분 4.90%를 미국의 글...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://ssl.pstatic.net/static.news/image/news...,23일 효성중공업은 조 회장이 상속세 재원을 마련하기 위해 시간 외 매매로 효성중 ...,"[{""stock_id"": ""298040"", ""stock_name"": ""효성중공업""}]","[{""stock_id"": ""004800"", ""stock_name"": ""효성""}, {...","[{""stock_id"": ""298040"", ""industry_id"": ""32801""...",0.08
4,20250523_0011,2025-05-23 17:52:00,몸집 키우는 카카오…'간편결제 빅3' 흔드나,"카카오페이, 신세계 페이 품나\n이커머스 결제 강화 점유 확대\n압도적 1위 네이버...",매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...,카카오페이가 신세계 이마트 측에서 쓱페이·스마일페이 인수를 추진하고 나선 건 국내 ...,"[{""stock_id"": ""377300"", ""stock_name"": ""카카오페이""}]","[{""stock_id"": ""377300"", ""stock_name"": ""카카오페이""}]","[{""stock_id"": ""377300"", ""industry_id"": ""116601...",0.33


In [4]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13720 entries, 0 to 13719
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   news_id          13720 non-null  object 
 1   wdate            13720 non-null  object 
 2   title            13720 non-null  object 
 3   article          13720 non-null  object 
 4   press            13720 non-null  object 
 5   url              13720 non-null  object 
 6   image            13720 non-null  object 
 7   summary          13720 non-null  object 
 8   stock_list       13720 non-null  object 
 9   stock_list_view  13720 non-null  object 
 10  industry_list    13720 non-null  object 
 11  impact_score     13720 non-null  float64
dtypes: float64(1), object(11)
memory usage: 1.3+ MB


In [5]:
df_data[df_data["news_id"] == '20250522_0073']

,news_id,wdate,title,article,press,url,image,summary,stock_list,stock_list_view,industry_list,impact_score
35,20250522_0073,2025-05-22 11:26:00,"""1억7940만원어치 샀다""…카카오 임원 자사주 매입 '우르르' [종목+]","카카오 임원 줄줄이 자사주 매입…주가 반등 나설까\n카카오 임원 8명, 총 1.8억...",한국경제,https://n.news.naver.com/mnews/article/015/000...,https://imgnews.pstatic.net/image/015/2025/05/...,카카오 임원들이 실적 부진으로 좀처럼 반등하지 못하는 상황에서 임원들이 책임 경영 ...,"[{""stock_id"": ""035720"", ""stock_name"": ""카카오""}]","[{""stock_id"": ""035720"", ""stock_name"": ""카카오""}]","[{""stock_id"": ""035720"", ""industry_id"": ""106301...",0.2


In [11]:
embedding = OnnxEmbedder(
    model_path="../../modelapi/models/kr_sbert_mean_onnx/kr_sbert.onnx",
    tokenizer_path="../../modelapi/models/kr_sbert_mean_onnx/tokenizer.json",
)

In [12]:
from langchain.vectorstores import Chroma
from langchain.schema import Document

# 2. Document 리스트로 변환
documents = []
for _, row in df_data.iterrows():
    doc = Document(
        page_content=row["title"] + " " + row["summary"],
        metadata={
            "news_id": row["news_id"],
            "wdate": row["wdate"],
            "title": row["title"],
            "press": row["press"],
            "url": row["url"],
            "image": row["image"],
            "impact_score": row["impact_score"],
            "stock_list": row["stock_list"],
            "industry_list": row["industry_list"],
        },
    )
    documents.append(doc)

# 4. Chroma 벡터DB 생성 (디스크 저장 가능)
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory="../../modelapi/db/chroma_store",  # 저장 위치
)

# 5. 저장
vectordb.persist()

KeyboardInterrupt: 

In [ ]:
from langchain.vectorstores import Chroma

vectordb = Chroma(
    persist_directory="../../modelapi/db/chroma_store",
    embedding_function=embedding,
)

/var/folders/pl/c9208lms3f937xrsflvwrm640000gn/T/ipykernel_11205/640673004.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


In [ ]:
# 검색
results = vectordb.similarity_search_with_score(
    "카카오 임원들이 실적 부진으로 좀처럼 반등하지 못하는 상황에서 임원들이 책임 경영 및 주주가치 제고를 위해 자사주 매입에 나섰으며 이는 저점 매수에 나선 것이라는 해석이 나오고 있다.   kksshin@hankookild.com",
    k=50,
)

for i, (doc, score) in enumerate(results):
    similarity = 1 - score
    print(f"\nTop {i+1}")
    print(f"유사도: {similarity:.4f}")
    print(f"요약: {doc.page_content}")
    print(f"기타: {doc.metadata}")


Top 1
유사도: 1.0000
요약: 카카오 임원들이 실적 부진으로 좀처럼 반등하지 못하는 상황에서 임원들이 책임 경영 및 주주가치 제고를 위해 자사주 매입에 나섰으며 이는 저점 매수에 나선 것이라는 해석이 나오고 있다.   kksshin@hankookild.com
기타: {'news_id': '20250522_0073', 'image': 'https://imgnews.pstatic.net/image/015/2025/05/22/0005135178_001_20250522114309510.jpg?type=w800', 'industry_list': '["자료처리, 호스팅, 포털 및 기타 인터넷 정보매개 서비스업"]', 'title': '"1억7940만원어치 샀다"…카카오 임원 자사주 매입 \'우르르\' [종목+]', 'press': '한국경제', 'impact_score': 0.2, 'article': '카카오 임원 줄줄이 자사주 매입…주가 반등 나설까\n카카오 임원 8명, 총 1.8억 규모 자사주 사들여\n"책임 경영 일환"\n사진=한경DB\n카카오 임원들이 자사주 매입에 나섰다. 주가가 실적 부진으로 좀처럼 반등하지 못하는 상황에서 임원들이 책임 경영에 대한 의지를 밝히는 동시에 주주 신뢰 회복에 나선 것으로 풀이된다.\n22일 금융감독원 전자공시시스템에 따르면 카카오 임원 8명은 지난 14일부터 최근까지 카카오 주식 1억7940만원어치를 장내서 매입했다. 가장 많이 매입한 임원은 이승현 성과리더다. 그는 지난 15~16일 카카오 140', 'stock_list': '["카카오"]', 'url': 'https://n.news.naver.com/mnews/article/015/0005135178', 'wdate': '2025-05-22 11:26:00'}

Top 2
유사도: 1.0000
요약: 카카오 임원들이 실적 부진으로 좀처럼 반등하지 못하는 상황에서 임원들이 책임 경영 및 주주가치 제고를 위해 자사주 매입에 나섰으며 이는 저점 매수에 나선 것이라는 해석

In [27]:
res = [{"wdate": doc.metadata['wdate']} for doc in results]

AttributeError: 'tuple' object has no attribute 'metadata'

In [8]:
res

[]

In [22]:
import pandas as pd

df_meta = pd.read_csv("../../db/news_2023_2025_metadata.csv")
df_meta.head()

,news_id,summary,stock_list,industry_list,impact_score
0,20250523_0002,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 국내 대표 전자결제사업자인 카...,"[""카카오페이""]","[""금융 지원 서비스업""]",0.33
1,20250523_0004,세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자(COO)인 존 월드...,"[""삼성전자""]","[""통신 및 방송 장비 제조업""]",0.05
2,20250523_0007,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 카카오페이가 SSG닷컴 쓱페이...,"[""카카오페이""]","[""금융 지원 서비스업""]",0.33
3,20250523_0010,23일 효성중공업은 조 회장이 상속세 재원을 마련하기 위해 시간 외 매매로 효성중 ...,"[""효성중공업""]","[""전동기, 발전기 및 전기 변환 · 공급 · 제어 장치 제조업""]",0.08
4,20250523_0011,카카오페이가 신세계 이마트 측에서 쓱페이·스마일페이 인수를 추진하고 나선 건 국내 ...,"[""카카오페이""]","[""금융 지원 서비스업""]",0.33


In [23]:
df = pd.read_csv("../../db/news_2023_2025_raw.csv")
df.head()

,news_id,wdate,title,article,press,url,image
0,20250523_0002,2025-05-23 18:52:00,"[단독] 카카오페이, 2500만 회원 쓱·스마일페이 품나…간편결제 시장 빅3 경쟁 후끈",매각가 5000억 안팎 달할듯\n결제시장 내 입지강화 포석\n카카오페이 [사진 = ...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...
1,20250523_0004,2025-05-23 18:33:00,"골드만삭스 차기 CEO, 이재용·김병주·이창용 만났다",[사진=연합뉴스]\n세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...
2,20250523_0007,2025-05-23 18:00:00,[단독] 전자결제 강자 카카오페이 쓱·스마일 페이 인수 추진,국내 대표 전자결제사업자인 카카오페이가 신세계이마트 산하 간편결제사업부 인수에 나섰...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://ssl.pstatic.net/static.news/image/news...
3,20250523_0010,2025-05-23 17:52:00,조현준 효성重 지분 4.9% 美 테크펀드 2600억 매각,조현준 효성 회장이 상속세 재원 마련을 위해 효성중공업 지분 4.90%를 미국의 글...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://ssl.pstatic.net/static.news/image/news...
4,20250523_0011,2025-05-23 17:52:00,몸집 키우는 카카오…'간편결제 빅3' 흔드나,"카카오페이, 신세계 페이 품나\n이커머스 결제 강화 점유 확대\n압도적 1위 네이버...",매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...


In [28]:
df_krx = pd.read_csv("../../db/KRX_KOSPI.csv", encoding="cp949")
df_krx.head()

,종목코드,종목명,종가,대비,등락률,상장시가총액
0,5930,삼성전자,55700,1800,3.34,329723832.0
1,660,SK하이닉스,208000,5500,2.72,151424492.0
2,207940,삼성바이오로직스,1029000,3000,0.29,73238046.0
3,373220,LG에너지솔루션,282500,10000,3.67,66105000.0
4,12450,한화에어로스페이스,884000,-3000,-0.34,41809842.0


In [29]:
df_krx["종목코드"] = df_krx["종목코드"].astype(str).str.zfill(6)

In [30]:
df_krx.head()

,종목코드,종목명,종가,대비,등락률,상장시가총액
0,005930,삼성전자,55700,1800,3.34,329723832.0
1,000660,SK하이닉스,208000,5500,2.72,151424492.0
2,207940,삼성바이오로직스,1029000,3000,0.29,73238046.0
3,373220,LG에너지솔루션,282500,10000,3.67,66105000.0
4,012450,한화에어로스페이스,884000,-3000,-0.34,41809842.0


In [31]:
import pandas as pd


# ③ 종목명 → 종목코드 딕셔너리 생성
name_to_code = dict(zip(df_krx["종목명"], df_krx["종목코드"]))

# ④ stock_list를 매핑된 딕셔너리 리스트로 변환하는 함수
import ast


def enrich_stock_list(stock_names_raw):
    try:
        stock_names = ast.literal_eval(stock_names_raw)  # 문자열 리스트 → 실제 리스트
        result = []
        for name in stock_names:
            code = name_to_code.get(name)
            if code:
                result.append({"stock_id": code, "stock_name": name})
        return result
    except Exception:
        return []


# ⑤ 적용
df_meta["stock_list"] = df_meta["stock_list"].apply(enrich_stock_list)

In [41]:
df_meta.iloc[15]['stock_list']

[{'stock_id': '003530', 'stock_name': '한화투자증권'},
 {'stock_id': '089590', 'stock_name': '제주항공'}]

In [42]:
df_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14087 entries, 0 to 14086
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   news_id        14087 non-null  object 
 1   summary        14087 non-null  object 
 2   stock_list     14087 non-null  object 
 3   industry_list  14087 non-null  object 
 4   impact_score   13749 non-null  float64
dtypes: float64(1), object(4)
memory usage: 550.4+ KB


In [44]:
df_krx_desc = pd.read_csv("../../db/KRX_KOSPI_DESCRIPTION.csv", encoding="cp949")
df_krx_desc.head()

,Unnamed: 0,종목코드,종목명,시장구분,소속부,소속부.1,업종코드,업종명,결산월,지정자문인,상장주식수,액면가,자본금,통화구분,대표이사,대표전화,주소
0,0,95570,AJ네트웍스,KOSPI,NaN,NaN,147603,산업용 기계 및 장비 임대업,12,NaN,45252759,1000,46822295000,원(KRW),곽정호,02-6363-9999,"서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)"
1,1,6840,AK홀딩스,KOSPI,NaN,NaN,116409,기타 금융업,12,NaN,13247561,5000,66237805000,원(KRW),"고준, 채형석 (각자 대표이사)",02-768-2923,서울특별시 마포구 양화로 188 -
2,2,27410,BGF,KOSPI,NaN,NaN,116409,기타 금융업,12,NaN,95716791,1000,95716791000,NaN,홍정국,1577-3663,서울특별시 강남구 테헤란로 405
3,3,282330,BGF리테일,KOSPI,NaN,NaN,74701,종합 소매업,12,NaN,17283906,1000,17283906000,원(KRW),민승배,02-1577-8007,서울특별시 강남구 테헤란로 405 BGF사옥
4,4,138930,BNK금융지주,KOSPI,NaN,NaN,116409,기타 금융업,12,NaN,318383519,5000,1629676230000,원(KRW),빈대인,051-620-3023,부산광역시 남구 문현금융로 30(문현동)


In [45]:
df_krx_desc["종목코드"] = df_krx_desc["종목코드"].astype(str).str.zfill(6)

In [53]:
import pandas as pd

# 2. 종목코드 6자리로 정제
df_krx_desc["종목코드"] = df_krx_desc["종목코드"].astype(str).str.zfill(6)

# 3. 종목코드 → 업종 매핑 딕셔너리 만들기
code_to_industry = {
    row["종목코드"]: {
        "industry_id": str(row["업종코드"]),
        "industry_name": row["업종명"],
    }
    for _, row in df_krx_desc.iterrows()
}


# 4. 뉴스에서 stock_list를 기반으로 industry_list 구성
def extract_industries(stock_list):
    industries = []
    seen = set()
    for stock in stock_list:
        stock_id = stock.get("stock_id")
        if stock_id is None:
            continue
        industry = code_to_industry.get(stock_id)
        if industry:
            key = (stock_id, industry["industry_id"])
            if key not in seen:
                seen.add(key)
                industries.append(
                    {
                        "stock_id": stock_id,
                        "industry_id": industry["industry_id"],
                        "industry_name": industry["industry_name"],
                    }
                )
    return industries


# 5. 뉴스 DataFrame 적용
df_meta["industry_list"] = df_meta["stock_list"].apply(extract_industries)

In [54]:
df_meta['industry_list'][0]

[{'stock_id': '377300',
  'industry_id': '116601',
  'industry_name': '금융 지원 서비스업'}]

In [56]:
df_meta.head()

,news_id,summary,stock_list,industry_list,impact_score
0,20250523_0002,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 국내 대표 전자결제사업자인 카...,"[{'stock_id': '377300', 'stock_name': '카카오페이'}]","[{'stock_id': '377300', 'industry_id': '116601...",0.33
1,20250523_0004,세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자(COO)인 존 월드...,"[{'stock_id': '005930', 'stock_name': '삼성전자'}]","[{'stock_id': '005930', 'industry_id': '32604'...",0.05
2,20250523_0007,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 카카오페이가 SSG닷컴 쓱페이...,"[{'stock_id': '377300', 'stock_name': '카카오페이'}]","[{'stock_id': '377300', 'industry_id': '116601...",0.33
3,20250523_0010,23일 효성중공업은 조 회장이 상속세 재원을 마련하기 위해 시간 외 매매로 효성중 ...,"[{'stock_id': '298040', 'stock_name': '효성중공업'}]","[{'stock_id': '298040', 'industry_id': '32801'...",0.08
4,20250523_0011,카카오페이가 신세계 이마트 측에서 쓱페이·스마일페이 인수를 추진하고 나선 건 국내 ...,"[{'stock_id': '377300', 'stock_name': '카카오페이'}]","[{'stock_id': '377300', 'industry_id': '116601...",0.33


In [77]:
df = pd.read_csv("../../db/news_2023_2025_metadata2.csv")
df.head()

,news_id,summary,stock_list,stock_list_view,industry_list,impact_score
0,20250523_0002,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 국내 대표 전자결제사업자인 카...,"[{'stock_id': '377300', 'stock_name': '카카오페이'}]",[],"[{'stock_id': '377300', 'industry_id': '116601...",0.33
1,20250523_0004,세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자(COO)인 존 월드...,"[{'stock_id': '005930', 'stock_name': '삼성전자'}]",[],"[{'stock_id': '005930', 'industry_id': '32604'...",0.05
2,20250523_0007,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 카카오페이가 SSG닷컴 쓱페이...,"[{'stock_id': '377300', 'stock_name': '카카오페이'}]",[],"[{'stock_id': '377300', 'industry_id': '116601...",0.33
3,20250523_0010,23일 효성중공업은 조 회장이 상속세 재원을 마련하기 위해 시간 외 매매로 효성중 ...,"[{'stock_id': '298040', 'stock_name': '효성중공업'}]",[],"[{'stock_id': '298040', 'industry_id': '32801'...",0.08
4,20250523_0011,카카오페이가 신세계 이마트 측에서 쓱페이·스마일페이 인수를 추진하고 나선 건 국내 ...,"[{'stock_id': '377300', 'stock_name': '카카오페이'}]",[],"[{'stock_id': '377300', 'industry_id': '116601...",0.33


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14087 entries, 0 to 14086
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   news_id          14087 non-null  object 
 1   summary          14087 non-null  object 
 2   stock_list       14087 non-null  object 
 3   stock_list_view  14087 non-null  object 
 4   industry_list    14087 non-null  object 
 5   impact_score     13749 non-null  float64
dtypes: float64(1), object(5)
memory usage: 660.5+ KB


In [79]:
# 변환 대상 컬럼
json_columns = ["stock_list", "stock_list_view", "industry_list"]


def fix_quotes(json_str):
    try:
        # 작은따옴표 → 객체 파싱
        obj = ast.literal_eval(json_str)
        # 객체 → 큰따옴표 JSON 문자열
        return json.dumps(obj, ensure_ascii=False)
    except Exception:
        return json_str  # 문제가 있으면 원래 값 유지


# 변환 적용
for col in json_columns:
    df[col] = df[col].astype(str).apply(fix_quotes)

In [80]:
df.head()

,news_id,summary,stock_list,stock_list_view,industry_list,impact_score
0,20250523_0002,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 국내 대표 전자결제사업자인 카...,"[{""stock_id"": ""377300"", ""stock_name"": ""카카오페이""}]",[],"[{""stock_id"": ""377300"", ""industry_id"": ""116601...",0.33
1,20250523_0004,세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자(COO)인 존 월드...,"[{""stock_id"": ""005930"", ""stock_name"": ""삼성전자""}]",[],"[{""stock_id"": ""005930"", ""industry_id"": ""32604""...",0.05
2,20250523_0007,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 카카오페이가 SSG닷컴 쓱페이...,"[{""stock_id"": ""377300"", ""stock_name"": ""카카오페이""}]",[],"[{""stock_id"": ""377300"", ""industry_id"": ""116601...",0.33
3,20250523_0010,23일 효성중공업은 조 회장이 상속세 재원을 마련하기 위해 시간 외 매매로 효성중 ...,"[{""stock_id"": ""298040"", ""stock_name"": ""효성중공업""}]",[],"[{""stock_id"": ""298040"", ""industry_id"": ""32801""...",0.08
4,20250523_0011,카카오페이가 신세계 이마트 측에서 쓱페이·스마일페이 인수를 추진하고 나선 건 국내 ...,"[{""stock_id"": ""377300"", ""stock_name"": ""카카오페이""}]",[],"[{""stock_id"": ""377300"", ""industry_id"": ""116601...",0.33


In [81]:
df.to_csv("../../db/news_2023_2025_metadata2.csv", index=False)